In [ ]:
import pickle
import numpy as np
import heatmaps
# from animated_plots import animateFFTRange, animateRangeHeatmap, animateDopplerRangeHeatmap, animateAzimuthRangeHeatmap
from fourier import rangeFFT, angleFFT, dopplerFFT
from params import PARAMS
from raw_signal import RadarData
import matplotlib.pyplot as plt
from time import sleep as sleep
from IPython.display import display, clear_output

In [ ]:
## Load files and get FRAMES
# Load data from file
data = pickle.load(open('data/openradar_25-01-23_QMUL-MCR.dat', 'rb'))
# Configure playback mode
config = data['config']
PARAMS.set_playback_mode(config)
# Read main parameters
rdata = RadarData(device='IWR6843ISK-ODS',
                  tx=PARAMS.TX_ANTENNAS,
                  rx=PARAMS.RX_ANTENNAS,
                  loops=PARAMS.CHIRP_LOOPS,
                  samples=PARAMS.ADC_SAMPLES)

# Get data frame
frames = data['data']


In [ ]:
## 1D-FFT example
from scipy.interpolate import CubicSpline
# Choose frame
adc_data = frames[2]
# Set RadarData raw_data
rdata.raw_data = adc_data
# Access data separated by Rx and Tx antennas
v_array = rdata.separated_vx_data
# 1D RANGE FFT and radar cube
[RC, rFFT, rBins] = rangeFFT(v_array[100,:,:],rdata.device)
# The range profile can be extracted from this as
rProf = np.mean(abs(rFFT[:,0:128]),axis=0)[8:]
# Cubic Spline
f = CubicSpline(rBins[8:128],rProf, bc_type='natural')
xnew= np.linspace(rBins[8],rBins[128],num=1000)
ynew = f(xnew)
# yinterp = np.interp(xvals, rBins[8:128], np.mean(abs(rFFT[:,0:128]),axis=0)[8:])
plt.plot(xnew,ynew)
plt.xlabel('Range (m)')
plt.grid(visible=True)
# plt.show()
plt.savefig('images/rProfile.png',dpi=600)


In [ ]:
## Animation range profile
fig, ax = plt.subplots()
line1 = ax.plot(0, 0)


ax.set_title('Range Profile')
ax.set_xlabel('Range (m)')
ax.grid(visible=True)


for n in range(np.size(frames,0)):
    # Choose frame
    adc_data = frames[n]
    # Set RadarData raw_data
    rdata.raw_data = adc_data
    # Access data separated by Rx and Tx antennas
    v_array = rdata.separated_vx_data
    ## 1D RANGE FFT and radar cube
    [RC, rFFT, rBins] = rangeFFT(v_array[100,:,:],rdata.device)
    # The range profile can be extracted from this as
    rProf = np.mean(abs(rFFT[:,0:128]),axis=0)[8:]
    # Cubic Spline
    f = CubicSpline(rBins[8:128],rProf, bc_type='natural')
    xnew= np.linspace(rBins[8],rBins[128],num=1000)
    ynew = f(xnew)

    ax.cla()
    ax.set_title('Range Profile')
    ax.set_xlabel('Range (m)')
    ax.grid(visible=True)
    ax.set_ylim(0,130000)
    ax.plot(xnew,ynew)
    ax.legend(['Frame #' + str(n+1)])
    display(fig)
    clear_output(wait = True)
    # fig.savefig('images/rProf'+str(n+1),dpi=600)
    plt.pause(0.1)

In [ ]:
## Animation range profile (1-frame)

fig, ax = plt.subplots()
line1 = ax.plot(0, 0)


ax.set_title('Range Profile')
ax.set_xlabel('Range (m)')
ax.grid(visible=True)

adc_data = frames[10]
# Set RadarData raw_data
rdata.raw_data = adc_data
# Access data separated by Rx and Tx antennas
v_array = rdata.separated_vx_data

for n in range(np.size(v_array,0)):
    ## 1D RANGE FFT and radar cube
    [RC, rFFT, rBins] = rangeFFT(v_array[n,:,:],rdata.device)
    # The range profile can be extracted from this as
    rProf = np.mean(abs(rFFT[:,0:128]),axis=0)[8:]
    # Cubic Spline
    f = CubicSpline(rBins[8:128],rProf, bc_type='natural')
    xnew= np.linspace(rBins[8],rBins[128],num=1000)
    ynew = f(xnew)

    ax.cla()
    # ax.set_label = 
    ax.set_ylim(0,130000)
    ax.plot(xnew,ynew)
    ax.legend(['Chirp #' + str(n+1)])
    ax.grid(visible=True)

    display(fig)
    clear_output(wait = True)
    # fig.savefig('images/rProfSlowTime'+str(n+1),dpi=600)
    plt.pause(0.1)

In [ ]:
## Phase evolution (1-frame)

fig, ax1 = plt.subplots()

ax1.set_title('Phase evolution in one frame')
ax1.set_xlabel('Chirp number')
ax1.set_ylabel('Phase (deg)')
ax1.grid(visible=True)
ax1.tick_params(axis='y',labelcolor='red')

ax2 = ax1.twinx()
ax2.set_ylabel('Target bin')
ax2.tick_params(axis ='y', labelcolor = 'blue') 

adc_data = frames[10]
# Set RadarData raw_data
rdata.raw_data = adc_data
# Access data separated by Rx and Tx antennas
v_array = rdata.separated_vx_data
## 1D RANGE FFT and radar cube
[RC, rFFT, rBins] = rangeFFT(v_array[0,:,:],rdata.device)
rMaxLoc = np.argmax(abs(rFFT[0,:128]))
# plt.plot(abs(rFFT[0,:128]))
# plt.show()
rPhase = np.zeros((1,np.size(v_array,0)))
rm = np.zeros((1,np.size(v_array,0)))

for n in range(np.size(v_array,0)):
    ## 1D RANGE FFT and radar cube
    [RC, rFFT, rBins] = rangeFFT(v_array[n,:,:],rdata.device)
    # The range profile can be extracted from this as
    rPhase[0,n] = np.angle(rFFT[0,rMaxLoc],deg=True)
    rm[0,n] = np.argmax(abs(rFFT[0,:128]))
ax2.plot(rm.T,color='blue')
ax1.plot(rPhase.T,color='red')
plt.show()
fig.savefig('images/phaseEvolution',dpi=600)


In [ ]:
# CAPON AND MUSIC ALGORITHM
import numpy as np
import capon as cp
# Choose frame
adc_data = frames[2]
# Set RadarData raw_data
rdata.raw_data = adc_data
# Access data separated by Rx and Tx antennas
v_array = rdata.separated_vx_data
# 1D RANGE FFT and radar cube
[RC, rFFT, rBins] = rangeFFT(v_array[0,:,:],rdata.device)
# Azimuth and elevations bins
[_, _, aBins, eBins] = angleFFT(RC)
# Choose the device
# dev = 'ISK'
dev = 'ODS'
# Antenna layout for steering vector
antLay = np.zeros((3,12))
if (dev == 'ISK'):    
    antLay[1,:] = np.dot([0,1,2,3,2,3,4,5,4,5,6,7],0.5)
    antLay[2,:] = np.dot([0,0,0,0,1,1,1,1,0,0,0,0],0.5)
elif (dev == 'ODS'):
    antLay[1,:] = np.dot([0,0,1,1,2,2,3,3,2,2,3,3],0.5)
    antLay[2,:] = np.dot([3,2,2,3,3,2,2,3,1,0,0,1],0.5)
# Steering vector
A = cp.steervec(antLay,1,aBins,eBins)

aMUSIC = []
for n in range(np.size(frames)):
# aCAPON = np.zeros((np.shape(rFFT)[1],np.shape(aBins)[0]*np.shape(eBins)[0]),dtype=complex)
# for kk in range(np.shape(rFFT)[1]):
#     aCAPON[kk,:],_ = cp.aoa_capon(rFFT,A.T)
    [_, rFFT, _] = rangeFFT(v_array[n,:,:],rdata.device)
    aMUSIC.append(cp.musicPS(rFFT,A))



In [ ]:
RAEHM = np.reshape(aMUSIC,(256,32,64))
fig, ax = plt.subplots()
c = plt.pcolormesh(aBins,rBins[:32],np.amax(abs(RAEHM[:32,:,:]),axis=1))
fig.colorbar(c,ax=ax)

In [ ]:
fig, ax = plt.subplots()
c = plt.pcolormesh(aBins, rBins[:32], aFFT[:,:32].T)
fig.colorbar(c,ax=ax)
plt.show()

In [ ]:
from numpy import linalg as LA
print
R = np.matmul(np.transpose(rFFT[:, 100][np.newaxis]),rFFT[:, 100][np.newaxis])
_, Qn = LA.eig(R)
Qn = np.delete(Qn,-1,1)
Qn = np.matmul(Qn,np.transpose(Qn))

PS = 1/np.diag(np.array(np.transpose(A) @ np.transpose(Qn) @ A))

In [ ]:
# Polar heatmaps
# heatmaps.plotAzimuthRangeHeatmap(rBins[:80],aBins[1:],aFFT[1:,:80])
# heatmaps.plotElevationRangeHeatmap(rBins[:80],eBins[1:],eFFT[1:,:80])
# Cartesian HEATMAP
from fourier import matlabMultip
# heatmaps.plotXYheatmap(rBins[:60],aBins[1:],10*np.log10(aFFT[1:,:60]))
ax = plt.axes()
X = matlabMultip(rBins[:60].T, np.sin(aBins[1:]*np.pi/180))
Y = matlabMultip(rBins[:60].T, np.cos(aBins[1:]*np.pi/180))
# ax.plot_surface(X, Y, 10*np.log10(aFFT[1:,:60]).T, cmap=plt.cm.jet,linewidth=0 )
ax.pcolormesh(X,Y,(aFFT[1:,:60]).T)
# ax.view_init(90, 90)

In [ ]:
## Doppler FFT
[dFFT, dBins, rBins] = dopplerFFT(v_array)

heatmaps.plotDopplerRangeHeatmap(rBins[:60],dBins,abs(dFFT[:,:60]))